##### Vadim BENICHOU
##### Jennifer VIAL


# Part I

1. We reiterated the steps of the wordcounts assignement.  

2. We made the HDFS steps to implement the data in HDFS. 

3. We created two text files : join1_FileA.txt and join1_FileB.txt

Here are our inputs in the join1_FileA:
able,991
about,11
burger,15
actor,22


Here are our inputs in the join1_FileB:
Jan-01 able,5
Feb-02 about,3
Mar-03 about,8
Apr-04 able,13
Feb-22 actor,3
Feb-23 burger,5
Mar-08 burger,2
Dec-15 able,100

Last but not least, we aim at using our map and reduce python functions on these two functions through the command cat join1_File*.txt | ./join1_mapper.py | sort | ./join1_reducer.py. 

## The code for the join1_mapper.py

In [1]:
#!/usr/bin/env python
import sys

for line in sys.stdin:
    line       = line.strip()   #strip out carriage return
    key_value  = line.split(",")   #split line, into key and value, returns a list
    key_in     = key_value[0].split(" ")   #key is first item in list
    value_in   = key_value[1]   #value is 2nd item 

    #print key_in
    if len(key_in)>=2:           #if this entry has <date word> in key
        date = key_in[0]      #now get date from key field
        word = key_in[1]
        value_out = date+" "+value_in     #concatenate date, blank, and value_in
        print( '%s\t%s' % (word, value_out) )  #print a string, tab, and string
    else:   #key is only <word> so just pass it through
        print( '%s\t%s' % (key_in[0], value_in) )  #print a string tab and string

#Note that Hadoop expects a tab to separate key value
#but this program assumes the input file has a ',' separating key value

## The code for the join1_reducer.py

In [2]:
#!/usr/bin/env python
import sys


prev_word          = "  "                #initialize previous word  to blank string
months             = ['Jan','Feb','Mar','Apr','Jun','Jul','Aug','Sep','Nov','Dec']

dates_to_output    = [] #an empty list to hold dates for a given word
day_cnts_to_output = [] #an empty list of day counts for a given word
# see https://docs.python.org/2/tutorial/datastructures.html for list details

line_cnt           = 0  #count input lines

for line in sys.stdin:
    line       = line.strip()       #strip out carriage return
    key_value  = line.split('\t')   #split line, into key and value, returns a list
    line_cnt   = line_cnt+1     

    #note: for simple debugging use print statements, ie:  
    curr_word  = key_value[0]         #key is first item in list, indexed by 0
    value_in   = key_value[1]         #value is 2nd item

    #-----------------------------------------------------
    # Check if its a new word and not the first line 
    #   (b/c for the first line the previous word is not applicable)
    #   if so then print out list of dates and counts
    #----------------------------------------------------
    if curr_word != prev_word:

        # -----------------------     
	#now write out the join result, but not for the first line input
        # -----------------------
        if line_cnt>1:
	    for i in range(len(dates_to_output)):  #loop thru dates, indexes start at 0
	         print('{0} {1} {2} {3}'.format(dates_to_output[i],prev_word,day_cnts_to_output[i],curr_word_total_cnt))
            #now reset lists
	    dates_to_output   =[]
            day_cnts_to_output=[]
        prev_word         =curr_word  #set up previous word for the next set of input lines

	
    # ---------------------------------------------------------------
    #whether or not the join result was written out, 
    #   now process the curr word    
  	
    #determine if its from file <word, total-count> or < word, date day-count>
    # and build up list of dates, day counts, and the 1 total count
    # ---------------------------------------------------------------
    if (value_in[0:3] in months): 

        date_day =value_in.split() #split the value field into a date and day-cnt
        
        #add date to lists of the value fields we are building
        dates_to_output.append(date_day[0])
        day_cnts_to_output.append(date_day[1])
    else:
        curr_word_total_cnt = value_in  #if the value field was just the total count then its
                                           #the first (and only) item in this list

# ---------------------------------------------------------------
#now write out the LAST join result
# ---------------------------------------------------------------
for i in range(len(dates_to_output)):  #loop thru dates, indexes start at 0
         print('{0} {1} {2} {3}'.format(dates_to_output[i],prev_word,day_cnts_to_output[i],curr_word_total_cnt))

To get the result, we enter the following commands in our terminal:
hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-input /user/cloudera/input \
-output /user/cloudera/join1_output \
-mapper /home/cloudera/join1_mapper.py \
-reducer /home/cloudera/join1_reducer.py

Thus in our join1_output.txt we get the following results:

Dec-15 able 100 991	
Apr-04 able 13 991	
Jan-01 able 5 991	
Mar-03 about 8 11	
Feb-02 about 3 11	
Feb-22 actor 3 22	
Feb-23 burger 5 15	
Mar-08 burger 2 15


# Part II

1. 

Firstly, by using make_join2data.py, we generated some datasets
through the following scripts:

python make_join2data.py y 1000 13 > join2_gennumA.txt 

python make_join2data.py y 2000 17 > join2_gennumB.txt

python make_join2data.py y 3000 19 > join2_gennumC.txt

python make_join2data.py n 100 23 > join2_genchanA.txt

python make_join2data.py n 200 19 > join2_genchanB.txt

python make_join2data.py n 300 37 > join2_genchanC.txt


Let's create the make_join2data script

In [ ]:
#!/usr/bin/env python
import sys

chans   = ['ABC','DEF','CNO','NOX','YES','CAB','BAT','MAN','ZOO','XYZ','BOB']
sh1 =['Hot','Almost','Hourly','PostModern','Baked','Dumb','Cold','Surreal','Loud']
sh2 =['News','Show','Cooking','Sports','Games','Talking','Talking']
vwr =range(17,1053)

chvnm=sys.argv[1]  #get number argument, if its n, do numbers not channels,

lch=len(chans)
lsh1=len(sh1)
lsh2=len(sh2)
lvwr=len(vwr)
ci=1
s1=2
s2=3
vwi=4
ri=int(sys.argv[3])
for i in range(0,int(sys.argv[2])):  #arg 2 is the number of lines to output

    if chvnm=='n':  #no numuber
        print('{0}_{1},{2}'.format(sh1[s1],sh2[s2],chans[ci]))
    else:
        print('{0}_{1},{2}'.format(sh1[s1],sh2[s2],vwr[vwi])) 
    ci=(5*ci+ri) % lch   
    s1=(4*s1+ri) % lsh1
    s2=(3*s1+ri+i) % lsh2
    vwi=(2*vwi+ri+i) % lvwr
 
    if (vwi==4): vwi=5

2.

We put this files in a HDFS directory:

join2_genchanA.txt

join2_genchanB.txt

join2_genchanC.txt

join2_gennumA.txt

join2_gennumB.txt

join2_gennumC.txt

4. 

Here, we have first to create new map / reduce python files, and use them to get the total number of viewers for shows on ABC.



Creation of our mapper: 

In [4]:
#!/usr/bin/env python
import sys

for line in sys.stdin:
    line       = line.strip()   #strip out carriage return
    key_value  = line.split(",")   #split line, into key and value, returns a list
    key_in     = key_value[0].split(" ")   #key is first item in list
    value_in   = key_value[1]   #value is 2nd item 

    #print key_in
    if (value_in == "ABC") or (value_in.isdigit()):           #if this entry equal 'ABC' or is a digit
        print( '%s\t%s' % (key_in[0], value_in) )  #print a string, tab, and string
   


In [6]:
#! /usr/bin/env python

import sys

prev_show = " "
abc_found = False

shows_output = []
viewers_output = []

line_cnt = 0

for line in sys.stdin:
	line = line.strip()
	key_value = line.split('\t')
	line_cnt = line_cnt+1

	curr_show = key_value[0]
	value_in = key_value[1]

	if curr_show != prev_show:
		
		if abc_found == True:
		
			shows_output.append(prev_show)
			viewers_output.append(views)

		views = 0	
		abc_found = False
		prev_show = curr_show
		
	if value_in.isdigit():
		views += int(value_in)

	else: 
		abc_found = True

if abc_found == True:
		
	shows_output.append(prev_show)
	viewers_output.append(views)

for i in range(len(shows_output)):
	print('{0} {1}'.format(shows_output[i], viewers_output[i]))
	


  0


Let's run the command bellow: 

hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar

-input /user/cloudera/input_join2/ 

-output /user/cloudera/total_viewer_counts_output 

-mapper /home/cloudera/join2_mapper.py 

-reducer /home/cloudera/join2_reducer.py --numReduceTasks 1


Almost_News 34491

Almost_Show 60103

Cold_Sports 62106

Dumb_Show 54926

Surreal_Sports 47935

Hourly_Show 18582

Dumb_Talking 114895	

Hot_Games 65241	

Hot_Show 84371	

Almost_Games 43247	

Hourly_Cooking 56309

Baked_News 57312

Hourly_Talking 118462

Baked_Games 61703

Loud_Show 51921

PostModern_Games 51654	

Loud_Games 48591

Surreal_News 63421	

Surreal_Talking 169459

PostModern_News 61071

Cold_News 48434	

# Part III

In this part, we aim at implementing the vector-vector multiplication. 

Let's make the vec mul mapper:

In [ ]:
#!/usr/bin/env python
import sys

for line in sys.stdin:
	line       = line.strip()   
	key_value  = line.split(",")   
	key_in     = key_value[0].strip()   
	value_in   = key_value[1].strip()   
	print( '%s\t%s' % (key_in, value_in) )

Let's make the vec mul reducer:

In [ ]:
#!/usr/bin/env python
import sys

prev.key = " "                
total.sum = 0
line.c = 0  

for line in sys.stdin:
    line       = line.strip()       
    key_value  = line.split('\t')   
    line.c   = line.c+1     

   
    curr_key  = key_value[0].strip()         
    value_in   = key_value[1].strip()         

    if line.c==1:
        prev_key= curr_key
        curr_total = float(value_in)
        continue

    if (curr_key != prev_key):
        total_sum+=curr_total
        
        curr_total = 0
        prev.key= curr_key
	
    # process the value
    if curr_total ==0:
        curr_total = float(value_in)
    else:
        curr_total *= float(value_in)
# ---------------------------------------------------------------
#now write out the LAST join result
# ---------------------------------------------------------------
total_sum+=curr_total
print('total_dot_product ',total_sum)


Now, we want to implement the matrix-vector mapper:

In [ ]:
#!/usr/bin/env python
import sys

# input_dir = sys.argv[1]
input_dir = '.'
vector = [float(v) for v in open(input_dir+'/vector_file').readlines()]

for line in sys.stdin:
	line       = line.strip()   
	key_value  = line.split(",")   
	i     = int(key_value[0].strip())   
	j   = int(key_value[1].strip())   
	value   = float(key_value[2].strip())
	print( '%s\t%s' % (i, value*vector[j]) )
  


Now, we want to implement the matrix-vector reducer:

In [ ]:
#!/usr/bin/env python
import sys

prev.key = " "                
total.sum = 0
line.c = 0 

for line in sys.stdin:
    line       = line.strip()       
    key_value  = line.split('\t')   
    line.c   = line.c+1     

    
    curr_key  = key_value[0].strip()         
    value_in   = key_value[1].strip()        

    if line.c==1:
        prev_key= curr_key
        curr_total = float(value_in)
        continue

    if (curr_key != prev_key):
        total_sum+=curr_total
        print( '%s\t%s' % (prev_key, curr_total) )
        
        curr_total = 0
        prev_key= curr_key
	
    # process the value
    curr_total += float(value_in)
# ---------------------------------------------------------------
#now write out the LAST join result
# ---------------------------------------------------------------
print( '%s\t%s' % (curr_key, curr_total) )
